In [ ]:
import numpy as np;
import numpy.random as rd;

import matplotlib.pyplot as plt;
import pyaudio;
import wave;
import scipy.io.wavfile as sciiowav;
import scipy.signal as scisig;

%matplotlib notebook

In [ ]:
A4   = 440.0; #Hz
RATE = 44100.0; #Hz 
VOL = 0.25;

#para el temperamento igual
def num2freq(n):
    return A4*np.power(2.0,(n-69.0)/12.0);

#onda sinusoidal
def get_sine(n,s,h=1):
    s_ = np.arange(int(RATE*s));
    return np.sin(2.0*np.pi*h*(num2freq(n)/RATE)*s_);

def get_sawtooth(n,s):
    s_ = np.arange(int(RATE*s));
    return scisig.sawtooth(2.0*np.pi*(num2freq(n)/RATE)*s_);

def get_triangle(n,s):
    s_ = np.arange(int(RATE*s));
    return 2.0*np.abs(scisig.sawtooth(2.0*np.pi*(num2freq(n)/RATE)*s_))-1.0;

def get_square(n,s):
    s_ = np.arange(int(RATE*s));
    return scisig.square(2.0*np.pi*(num2freq(n)/RATE)*s_);

In [ ]:
WIN_SIZE = 8192;

t_win = np.arange(WIN_SIZE)/RATE;
print u"Duración de la ventana ~ " + str(WIN_SIZE/RATE) + "s"; 
gaussian_window = scisig.windows.gaussian(WIN_SIZE,np.sqrt(WIN_SIZE));

win_arr = np.zeros((3,3,WIN_SIZE));
win_nam = [["","",""],["","",""],["","",""]];

win_nam[0][0]  = "Hamming";
win_arr[0,0,:] = scisig.windows.hamming(WIN_SIZE);

win_nam[1][0]  = "Hann";
win_arr[1,0,:] = scisig.windows.hann(WIN_SIZE);

win_nam[2][0]  = "Triangular";
win_arr[2,0,:] = scisig.windows.triang(WIN_SIZE);

win_nam[0][1]  = "Flattop";
win_arr[0,1,:] = scisig.windows.flattop(WIN_SIZE);

win_nam[1][1]  = "Blackman";
win_arr[1,1,:] = scisig.windows.blackman(WIN_SIZE);

win_nam[2][1]  = "Coseno";
win_arr[2,1,:] = scisig.windows.cosine(WIN_SIZE);

win_nam[0][2]  = "Bohman";
win_arr[0,2,:] = scisig.windows.bohman(WIN_SIZE);

win_nam[1][2]  = "Bartlett";
win_arr[1,2,:] = scisig.windows.bartlett(WIN_SIZE);

win_nam[2][2]  = "Nuttall";
win_arr[2,2,:] = scisig.windows.nuttall(WIN_SIZE);

fig,ax = plt.subplots(3,3,figsize=(12.8,12.8),sharex=True);
fig.suptitle("Funciones Ventana para la STFT",fontsize=16);
for i in range(3):
    for j in range(3):
        ax[i,j].set_title(win_nam[i][j]);
        ax[i,j].plot(t_win,win_arr[i,j,:]);

In [ ]:
modos = {};
modos[u'lidia']     = [[ 0, 2, 4, 6, 7, 9,11,12],[ 0, 4, 7]];
modos[u'jónica']    = [[ 0, 2, 4, 5, 7, 9,11,12],[ 0, 4, 7]];
modos[u'mixolidia'] = [[ 0, 2, 4, 5, 7, 9,10,12],[ 0, 4, 7]];
modos[u'doria' ]    = [[ 0, 2, 3, 5, 7, 9,10,12],[ 0, 3, 7]];
modos[u'éolica']    = [[ 0, 2, 3, 5, 7, 8,10,12],[ 0, 3, 7]];
modos[u'frigia']    = [[ 0, 1, 3, 5, 7, 8,10,12],[ 0, 3, 7]];
modos[u'locria']    = [[ 0, 1, 3, 5, 6, 8,10,12],[ 0, 3, 6]];

#Acomodado por su brillo
nmodos = [u'lidia',
          u'jónica',
          u'mixolidia',
          u'doria' ,
          u'éolica',
          u'frigia',
          u'locria'];

def get_sound(nom,o,get_f):
    scala,prin = modos[nom];
    sonido = np.zeros((0,));
    for n in scala:
        sonido = np.r_[sonido,get_f(n+12*o,0.5)];
    p = get_f(prin[0]+12*o,2.0);
    for i in prin[1:]:
        p += get_f(i+12*o,2.0);
    sonido = np.r_[sonido,p];
    p += get_f(scala[-2]+12*o,2.0);
    sonido = np.r_[sonido,p];
    return sonido;

In [ ]:
idx = 6;
sonido = get_sound(nmodos[idx],5,get_square);
sonido += 1e-1*rd.randn(sonido.shape[0]);
print "Duración de la escala ~ " + str(sonido.shape[0]/RATE) + "s a " + str(RATE) + " Hz";

t0 = 0.25;
t0idx = int(t0*RATE); 

i = 1;
j = 1; 

cacho_sonido = sonido[t0idx:t0idx+WIN_SIZE];

ventana_sonido = cacho_sonido*win_arr[i,j,:];

fig,ax = plt.subplots(2,1,figsize=(12.8,12.8),sharex=True);
fig.suptitle("Aplicando una Ventana de "+ win_nam[i][j] + " a sonido",fontsize=16);

ax[0].set_title("Sonido sin ventana");
ax[0].plot(t_win+t0,cacho_sonido);
ax[1].set_title("Sonido con ventana de "+ win_nam[i][j]);
ax[1].plot(t_win+t0,ventana_sonido),ax[1].plot(t_win+t0,win_arr[i,j,:],'k--');




In [ ]:
ventana_fft = np.fft.rfft(ventana_sonido);
ventana_abs = np.abs(ventana_fft);

f_win = np.linspace(0.0,0.5*RATE,WIN_SIZE/2); 

fig,ax = plt.subplots(1,1,figsize=(12.8,12.8),sharex=True);
fig.suptitle("Aplicando la Transformada de Fourier a sonido con ventana",fontsize=16);

ax.set_xlabel(u'Frecuencia (Hz)');
ax.set_ylabel(u'Potencia (dB)');
ventana_max = np.max(ventana_abs[:-1]);
ax.plot(f_win,np.log10((ventana_abs[:-1]+1e-12)/(ventana_max+1e-12)),'k--');

In [ ]:
def get_spectrogram(x,win,step):
    win_size   = win.shape[0];
    win_size_2 = win.shape[0]/2;

    espectro_t = np.zeros((0,));
    espectro_x = np.zeros((win_size_2,0));
    
    j = 0;
    for i in range(0,sonido.shape[0],step):
        espectro_t = np.r_[espectro_t,i/RATE];
        aux_x = np.zeros((0,));
        if j % 10 == 0:
            print i/RATE;
        aux_x = np.r_[np.zeros((max(win_size_2-i,0),)),
                      x[max(0,i-win_size_2):min(x.shape[0],i+win_size_2)],
                      np.zeros((max(i+win_size_2-x.shape[0],0),))];
        aux_abs = np.abs(np.fft.rfft(win*aux_x))[:-1];
        espectro_x = np.c_[espectro_x,aux_abs];
        j = j + 1;
    return espectro_t,espectro_x;


In [ ]:
STEP_SIZE = WIN_SIZE/4;
espectro_t,espectro_sonido = get_spectrogram(sonido,win_arr[i,j,:],STEP_SIZE);

In [ ]:
Ctidx = int(num2freq(120)*WIN_SIZE/RATE+1);

frec = np.linspace(0.0,0.5*RATE,WIN_SIZE/2);

print "Duración de la ventana Blackman ~ " + str(WIN_SIZE/RATE) + "s";

fig,ax = plt.subplots(1,1,figsize=(12.8,9.6),sharex=True);
fig.suptitle("STFT del sonido con ventana de " + win_nam[i][j],fontsize=16);

ax.set_ylabel(u'Frecuencia (Hz)');
ax.set_yticks(np.arange(0,Ctidx,128));
ax.set_yticklabels(frec[:Ctidx:128]);
ax.set_xlabel(u'Tiempo (s)');
ax.set_xticks(np.arange(0,espectro_t.shape[0],32));
ax.set_xticklabels(espectro_t[::32]);

espectro_max = np.max(espectro_sonido);

ax.imshow(np.log10((espectro_sonido[:Ctidx,:]+1e-20)/(espectro_max+1e-20)),cmap='gnuplot2',aspect='auto');

In [ ]:
def get_chroma_vector(X,o):
    C = np.zeros((12,));
    for n in range(12):
        for h in range(1,7):
            k_l = int(np.floor(h*num2freq(n+12*o-0.125)*WIN_SIZE/RATE));
            k_h = int(np.ceil(h*num2freq(n+12*o+0.125)*WIN_SIZE/RATE));
            Xm = np.max(X[k_l:k_h]);
            C[n] += Xm/h;
            #u = (k_h-k_l);
            #C[n] += (u*X[k_l]+(1.0-u)*X[k_h])/h;
    return C;


In [ ]:
c_win = [get_chroma_vector(ventana_abs,o) for o in range(3,6)];
colors = ['green','turquoise','blue','indigo','violet','purple','red','orangered','orange','yellow','lime'];
fig,ax = plt.subplots(3,1,figsize=(12.8,9.6),sharex=True);
fig.suptitle("Cromagramas para octavas 3,4 y 5",fontsize=16);
ax[0].set_xticks(np.arange(12));
ax[0].set_xticklabels(['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']);

ax[0].set_ylabel("Presencia para octava 3");
ax[1].set_ylabel("Presencia para octava 4");
ax[2].set_ylabel("Presencia para octava 5");

ax[2].set_xlabel(u'Notas');

ax[0].bar(np.arange(12),c_win[0],color=colors);
ax[1].bar(np.arange(12),c_win[1],color=colors);
ax[2].bar(np.arange(12),c_win[2],color=colors);


In [ ]:
def get_chromagram(sx,o):
    cx = np.zeros((12,sx.shape[1]));
    for t in range(sx.shape[1]):
        cx[:,t] = get_chroma_vector(sx[:,t],o);
    return cx;


In [ ]:
c_sonido = (get_chromagram(espectro_sonido,3)
           +get_chromagram(espectro_sonido,4)
           +get_chromagram(espectro_sonido,5)
           +get_chromagram(espectro_sonido,6))/4.0;

fig,ax = plt.subplots(1,1,figsize=(12.8,9.6),sharex=True);
fig.suptitle("Cromagrama del sonido con ventana de " + win_nam[i][j],fontsize=16);

ax.set_ylabel(u'Nota');
ax.set_yticks(np.arange(12));
ax.set_yticklabels(['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']);
ax.set_xlabel(u'Tiempo (s)');
ax.set_xticks(np.arange(0,espectro_t.shape[0],16));
ax.set_xticklabels(espectro_t[::16]);

ax.imshow(c_sonido,cmap='gnuplot2',aspect='auto');